In [1]:
import sys, os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal
import importlib
import h5py
sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [2]:
path = "/Users/ajc/FUSE/simulations/production/barrow_watershed"

var_set = ['surface_flux-day.dat']

In [3]:
S1 = os.path.join(path, 'CESM_8/CESM_5050_R10_B')
S2 = os.path.join(path, 'CESM_8/CESM_5050_R10_BY')
S3 = os.path.join(path, 'CESM_8/CESM_5050_R10_BZ')
S4 = os.path.join(path, 'CESM_8/CESM_5050_R10_BZZ')

Simulations = [[S1],[S2],[S3],[S4]]

In [4]:
S1 = os.path.join(path, 'CESM_9/CESM_5050_R10_B1')
S2 = os.path.join(path, 'CESM_9/CESM_5050_R10_B2')
S3 = os.path.join(path, 'CESM_9/CESM_5050_R10_B3')
S4 = os.path.join(path, 'CESM_9/CESM_5050_R10_B4')
#print (S1)
Simulations = [[S1],[S2],[S3],[S4]]

In [5]:
S1 = os.path.join(path, 'CESM_10/CESM_5050_R10_B')
S2 = os.path.join(path, 'CESM_10/CESM_5050_R10_BX')
S3 = os.path.join(path, 'CESM_10/CESM_5050_R10_BY')
#print (S1)
Simulations = [[S1],[S2],[S3]]

In [6]:
def read_file(file):
    d = np.loadtxt(file,usecols=1,skiprows=6)
    t1 = np.loadtxt(file,usecols=0,skiprows=6)
    t = np.array([x/86400./365. for x in t1])
    #print (len(d),len(t))
    l = np.minimum(len(d),len(t))
    return d[:l],t[:l]

def read_set(set,var):
    dat = []
    tim = []
    for s in set:
        file = os.path.join(path,s, var)
        #print (file)
        d = np.loadtxt(file,usecols=1,skiprows=6)
        t1 = np.loadtxt(file,usecols=0,skiprows=6)
        t = np.array([x/86400./365. for x in t1])
        
        L = len(d)%365
        if L >0:
            d = d[:-L]
            t = t[:len(d)]
            
        dat.append(d)
        tim.append(t)
    dat = np.concatenate(dat)
    tim = np.concatenate(tim)
    #print (len(dat)%365, len(tim)%365,L)
    
    return dat, tim

In [ ]:

path_sf=os.path.join(S1,"visdump_surface_star_data.h5")
print (S1,path_sf)
file_surf= h5py.File(path_sf,'r')
#basin_area = np.sum(list(file_surf['surface_star-cell_volume.cell.0']['00000']))

In [ ]:
forcing = h5py.File("/Users/ajc/FUSE/simulations/production/barrow_watershed/data/CESM/CESM-RCP8_5-2006-2100_snowadjusted_supersmoothDM_2006.h5",'r')

In [ ]:
d1_rain = forcing['precipitation rain [m s^-1]']
d2_snow = forcing['precipitation snow [m SWE s^-1]']
rain_mm = np.reshape(d1_rain,(-1,365)).mean(axis=1)*86400*365*1000
snow_mm = np.reshape(d2_snow,(-1,365)).mean(axis=1)*86400*365*1000
precip_annual = snow_mm + rain_mm

In [ ]:
key_var = ['surface_star-evaporative_flux','surface_star-depth']
def getSurfaceData(i,var):
    Data = []
    print (var)
    for s in Simulations[i]:
        infile = os.path.join(s,'visdump_surface_star_data.h5')
        print (infile)
        dx=h5py.File(infile,'r')

        d = np.array(readdata.GetSurfVarFromVis_All(infile,var)[var])
        d = d.mean(axis=1)

        L = len(d)%365
        if L >0:
            d = d[:-L]
            
        Data.append(d)
        print (d.shape, len(d), L)
    d = dict()
    d[var] = np.concatenate(Data)
    
    
    return d

In [ ]:
DataEvap = dict()
Slabels = ['S1', 'S2','S3','S4','S6']
for i in range (4):
    d = getSurfaceData(i, key_var[0])
    DataEvap[Slabels[i] + '_' + key_var[0]] = d[key_var[0]]


In [ ]:
DataEvap.keys()
#DataEvap['S1_surface_star-evaporative_flux']

In [7]:
colors = ['r','k','g','m','c','y']
marks = ['.','*', '+', 'o']
marks = ['-','-','-']
Labels = ['DM', r'S$_{1}$', r'S$_{2}$', r'S$_{3}$',r'S$_{4}$',r'S$_{6}$']

In [14]:
def plot_discharge_only():
    %matplotlib qt
    fig, axs = plt.subplots(1,2, figsize=(8,6), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    axs = axs.ravel()
    Colors = ['r','k','g','c','r']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_year_mm = 86400*365 * 1000
    basin_area = 112635.48
    to_mm = 1000.
    Labels = ['n = 1.0', 'n = 0.1','n = 0.05'] # 5 years
    
    for i,sim in enumerate(Simulations):
        Q_annual_avg = []
        var = var_set[0]
        d1, t1 = read_set(sim,var)
        d1 = [x/(55555.*basin_area)*86400*1000 for x in d1]
        
        axs[0].plot(d1,color=Colors[i],label=Labels[i])

        axs[0].set_xlabel('DoY',fontsize=12, fontweight='normal')
        axs[0].set_ylabel('Discharge [mm/day]',fontsize=12, fontweight='normal')
        d2 = np.reshape(d1,(-1,365)).mean(1)*365
        Q_annual_avg.append(d2)
        #print (d2,Q_annual_avg)
        #axs[1].bar(np.arange(len(Q_annual_avg)), Q_annual_avg,width=0.2)
        axs[1].plot(d2,color=Colors[i],label=Labels[i])
        axs[1].set_ylim((0,15))
    #axs[1].set_xticks(np.arange(4))
    #axs[1].set_xticklabels(Labels)
    #axs[1].set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')
    axs[0].legend()

    fig.tight_layout()
    #plt.savefig(outfile+'Q-ET-Mannings.pdf', bbox_inches='tight', dpi=800)

In [16]:
plot_discharge_only()

In [ ]:
outfile='/Users/ajc/Core/PostProcessing/2020/barrow_watershed/projections_02/Debug/'

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(2,2, figsize=(8,6), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
    
plt.tight_layout()
axs = axs.ravel()
Colors = ['r','k','g','c','r']
time = 2006 + np.linspace(0,94,94,dtype='int')
to_year_mm = 86400*365 * 1000
basin_area = 112635.48
to_mm = 1000.
Labels = ['n = 10', 'n = 1', 'n = 0.1', 'n = 0.05'] # 1 year
Labels = ['n = 0.5', 'n = 1.', 'n = 2.5', 'n = 5'] # 5 years
Q_annual_avg = []
E_annual_avg = []
kk=4
s, e = 0*365, (kk+1)*365
for i,sim in enumerate(Simulations):
    var = var_set[0]
    d1, t1 = read_set(sim,var)
    d1 = [x/(55555.*basin_area)*86400*1000 for x in d1[s:e]]
    #print (len(d1))
    axs[0].plot(d1,color=Colors[i],label=Labels[i])#,marker='-')
    
    axs[0].set_xlabel('DoY',fontsize=12, fontweight='normal')
    axs[0].set_ylabel('Discharge [mm/day]',fontsize=12, fontweight='normal')
    #axs[0].set_xlim(140, 160)
    #axs[0].text(141.1, 3.5, 'n = Manning\'s coefficent')
    Keys = list(DataEvap.keys())
    #-d3 = DataEvap[Keys[i]][s:e]*86400*1000
    #axs.plot(d1_rain[:365]*86400*1000,color=Colors[i+2],label='Rain')
    #axs.plot(d2_snow[:365]*86400*1000,color=Colors[i+3],label='Snow')
    #-axs[1].plot(d3,color=Colors[i],label='Evaporation',linestyle='dashed')
    #axs[1].set_xlim(120, 220)
    axs[1].set_ylim(0, 8)
    axs[1].set_ylabel('Evaporation [mm/day]',fontsize=12, fontweight='normal')
    #d4 = S1_Depth['surface_star-depth']*100
    #axs.plot(d4,color=Colors[i+1],label='Snow depth')
    #axs.set_ylabel('Snow depth [cm]',fontsize=12, fontweight='normal')
    #-print (np.mean(d1)*365, np.mean(d3)*365)
    #-Q_annual_avg.append(np.mean(d1)*365)
    #-E_annual_avg.append(np.mean(d3)*365)
axs[2].bar(np.arange(4), Q_annual_avg,width=0.2)
axs[2].set_xticks(np.arange(4))
axs[2].set_xticklabels(Labels)
axs[2].set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')
axs[0].legend()

axs[3].bar(np.arange(4), E_annual_avg,width=0.2)
axs[3].set_xticks(np.arange(4))
axs[3].set_xticklabels(Labels)
axs[3].set_ylabel('Annual average evaporation [mm]',fontsize=12, fontweight='normal')

fig.tight_layout()
#plt.savefig(outfile+'Q-ET-Mannings.pdf', bbox_inches='tight', dpi=800)
#plt.savefig(outfile+'snowdepth.pdf', bbox_inches='tight', dpi=800)

In [ ]:
print (precip_annual[0])